In [1]:
import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import tkinter as tk
from tkinter import ttk
import folium
import webbrowser
import tempfile
import pandas as pd
from sklearn.neighbors import BallTree

# API-Token für Zugriff auf OpenRouteService
secrets = {
    'api_key': '5b3ce3597851110001cf624812d4898347ee4900b2b0e0eac8974a33'
}

# Pfad zur CSV-Datei mit den Ladesäulenstationen
csv_file_path = "/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/3.gold/charger.csv"

# Laden der Daten aus der CSV-Datei
data = pd.read_csv(csv_file_path)

# Funktion zur Berechnung der Entfernung zwischen zwei Koordinaten
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Funktion zur Berechnung des Mittelpunkts zwischen zwei Koordinaten
def calculate_midpoint(coord1, coord2):
    mid_lat = (coord1[0] + coord2[0]) / 2
    mid_lon = (coord1[1] + coord2[1]) / 2
    return mid_lat, mid_lon

# Funktion zur Berechnung der Gesamtdistanz
def calculate_total_distance(coordinates):
    total_distance = 0
    for i in range(1, len(coordinates)):
        coord1 = coordinates[i - 1]
        coord2 = coordinates[i]
        segment_distance = geodesic(coord1, coord2).kilometers
        total_distance += segment_distance
    return total_distance

# Funktion zur Geocodierung von Adressen
def geocode_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None

# Funktion zur Suche nach Ladestationen in der Nähe eines Punkts
def find_ladestationen_in_der_naehe(lat, lon, k=5):
    coordinates = data[['latitude', 'longitude']].values
    tree = BallTree(coordinates)

    # Die k nächsten Ladestationen zum gegebenen Punkt finden
    ladestationen_idx = tree.query([[lat, lon]], k=k, return_distance=False)[0]
    ladestationen = data.iloc[ladestationen_idx]
    return ladestationen

# Funktion zum Hinzufügen von Markern für Ladestationen auf der Karte
def add_ladestation_marker(map_obj, lat, lon, name, distance_to_point):
    popup_text = f"Ladestation\nName: {name}<br>Entfernung zum Punkt: {distance_to_point:.2f} km"
    folium.Marker([lat, lon], icon=folium.Icon(color='purple'), popup=popup_text).add_to(map_obj)


# Funktion zum Anzeigen der Route und der Ladestationen
def calculate_route():
    start_lat = float(entry_start_lat.get())
    start_lon = float(entry_start_lon.get())
    end_lat = float(entry_end_lat.get())
    end_lon = float(entry_end_lon.get())
    reichweite_km = float(entry_reichweite.get())

    # Karte erstellen
    m = folium.Map(location=[start_lat, start_lon], zoom_start=10)

    # Marker für Start- und Zielpunkt hinzufügen
    folium.Marker([start_lat, start_lon], icon=folium.Icon(color='blue'), popup='Startpunkt').add_to(m)
    folium.Marker([end_lat, end_lon], icon=folium.Icon(color='red'), popup='Zielpunkt').add_to(m)

    # Mittelpunkt zwischen Start- und Zielpunkt berechnen
    midpoint_lat, midpoint_lon = calculate_midpoint((start_lat, start_lon), (end_lat, end_lon))

    # Ladestationen in der Nähe des Startpunkts finden und auf der Karte anzeigen
    start_ladestationen = find_ladestationen_in_der_naehe(start_lat, start_lon, k=5)
    for _, ladestation in start_ladestationen.iterrows():
        distance = calculate_distance((ladestation['latitude'], ladestation['longitude']), (start_lat, start_lon))
        add_ladestation_marker(m, ladestation['latitude'], ladestation['longitude'], ladestation['Betreiber'], distance)

    # Ladestationen in der Nähe des Zielpunkts finden und auf der Karte anzeigen
    end_ladestationen = find_ladestationen_in_der_naehe(end_lat, end_lon, k=5)
    for _, ladestation in end_ladestationen.iterrows():
        distance = calculate_distance((ladestation['latitude'], ladestation['longitude']), (end_lat, end_lon))
        add_ladestation_marker(m, ladestation['latitude'], ladestation['longitude'], ladestation['Betreiber'], distance)

    # Ladestationen in der Nähe des Mittelpunkts finden und auf der Karte anzeigen
    midpoint_ladestationen = find_ladestationen_in_der_naehe(midpoint_lat, midpoint_lon, k=5)
    for _, ladestation in midpoint_ladestationen.iterrows():
        distance_to_start = calculate_distance((ladestation['latitude'], ladestation['longitude']), (start_lat, start_lon))
        distance_to_end = calculate_distance((ladestation['latitude'], ladestation['longitude']), (end_lat, end_lon))
        min_distance = min(distance_to_start, distance_to_end)
        add_ladestation_marker(m, ladestation['latitude'], ladestation['longitude'], ladestation['Betreiber'], min_distance)


    # Route von OpenRouteService abrufen
    api_url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={secrets['api_key']}&start={start_lon},{start_lat}&end={end_lon},{end_lat}"
    response = requests.get(api_url)

    if response.status_code == 200:
        route_data = response.json()
        # Koordinaten der Route extrahieren
        route_coordinates = [(coord[1], coord[0]) for coord in route_data["features"][0]["geometry"]["coordinates"]]

        # Linienobjekt für die Route erstellen
        route_line = folium.PolyLine(route_coordinates, color="blue", weight=5, opacity=0.7)
        route_line.add_to(m)

        # Gesamtdistanz berechnen (hier als Beispiel die Luftlinie zwischen Start und Ziel)
        gesamtdistanz_km = calculate_distance((start_lat, start_lon), (end_lat, end_lon))

        # Karte speichern und in einem Webbrowser anzeigen
        tmpfile = tempfile.NamedTemporaryFile(delete=False, suffix=".html")
        m.save(tmpfile.name)
        webbrowser.open(f"file://{tmpfile.name}", new=1)

        # Ausgabe der Gesamtdistanz
        distance_label.config(text=f"Gesamtdistanz zwischen Start- und Zielpunkt: {gesamtdistanz_km:.2f} km")

    else:
        distance_label.config(text="Fehler beim Abrufen der Routendaten.")

# Tkinter-Fenster erstellen
root = tk.Tk()
root.title("Route und Ladestationen")

# Eingabefelder für Start- und Zielpunkt und Reichweite erstellen
frame = ttk.Frame(root)
frame.pack(padx=10, pady=10)

ttk.Label(frame, text="Startpunkt (Breitengrad):").grid(row=0, column=0, padx=5, pady=5)
ttk.Label(frame, text="Startpunkt (Längengrad):").grid(row=1, column=0, padx=5, pady=5)
ttk.Label(frame, text="Zielpunkt (Breitengrad):").grid(row=2, column=0, padx=5, pady=5)
ttk.Label(frame, text="Zielpunkt (Längengrad):").grid(row=3, column=0, padx=5, pady=5)
ttk.Label(frame, text="Reichweite (in km):").grid(row=4, column=0, padx=5, pady=5)

entry_start_lat = ttk.Entry(frame)
entry_start_lat.grid(row=0, column=1, padx=5, pady=5)
entry_start_lon = ttk.Entry(frame)
entry_start_lon.grid(row=1, column=1, padx=5, pady=5)
entry_end_lat = ttk.Entry(frame)
entry_end_lat.grid(row=2, column=1, padx=5, pady=5)
entry_end_lon = ttk.Entry(frame)
entry_end_lon.grid(row=3, column=1, padx=5, pady=5)
entry_reichweite = ttk.Entry(frame)
entry_reichweite.grid(row=4, column=1, padx=5, pady=5)

# Schaltfläche zum Berechnen der Route und Anzeigen der Ladestationen erstellen
calculate_button = ttk.Button(frame, text="Route und Ladestationen anzeigen", command=calculate_route)
calculate_button.grid(row=5, columnspan=2, padx=5, pady=10)

# Label für die Ausgabe der Gesamtdistanz erstellen
distance_label = ttk.Label(frame, text="")
distance_label.grid(row=6, columnspan=2, padx=5, pady=10)

root.mainloop()


: 